In [1]:
import AdventOfCode

In [2]:
input <- dayString 20

In [3]:
data Path a = Path [Segment a]
    deriving (Eq, Show)

data Segment a
    = Leaf a
    | Node [Path a]
    deriving (Eq, Show)

In [4]:
{-# LANGUAGE ScopedTypeVariables #-}
import Data.List

parseLeaf str = let
    (t, rest) = span (`elem` "NEWS") str
    in (Leaf t, rest)

parseNode acc ('(':str) = let
    (t, rest) = parsePath [] str
    in parseNode (acc++[t]) rest
parseNode acc ('|':str) = let
    (t, rest) = parsePath [] str
    in parseNode (acc++[t]) rest
parseNode acc (')':str) = (Node acc, str)

parsePath :: [Segment String] -> String -> (Path String, String)
parsePath acc []  = (Path acc, [])
parsePath acc str@(x:xs) = case x of
    '(' -> let
        (group, str') = parseNode [] str
        in parsePath (acc++[group]) str'
    '|' -> (Path acc, str)
    ')' -> (Path acc, str)
    _ -> let
        (chunk, str') = parseLeaf str
        in parsePath (acc++[chunk]) str'

parsePath [] "NSW(EE|NN|)S"

(Path [Leaf "NSW",Node [Path [Leaf "EE"],Path [Leaf "NN"],Path []],Leaf "S"],"")

In [5]:
trimmed = init $ tail input
processed = fst $ parsePath [] trimmed

In [6]:
maxPath (Path ts) = sum $ map maxSegment ts
maxSegment (Leaf str) = length str
maxSegment (Node rs) = let
    subpaths = map maxPath rs
    in if 0 `elem` subpaths then 0 else maximum subpaths

maxPath processed

3958

In [19]:
processed

Path [Leaf "WSSSEESESWSWSSWNNNENWWWWNNWWNWSSWNNNEEENEESE",Node [Path [Leaf "NENEENEESENENEEESWWSWSEENEESENNWNENENNEESSW",Node [Path [Leaf "SWSESSW",Node [Path [Leaf "N"],Path [Leaf "SESSWSSEEENNEESSENESEENNW",Node [Path [Leaf "NNWWS",Node [Path [Leaf "ESWENW"],Path []],Leaf "WWNNNENWWN",Node [Path [Leaf "EEENEEEENENWNENESEESSESWWSESENENENWNWNNESENNNNEEEENNNWNNESESSESSSW",Node [Path [Leaf "WWWWSEESWWSS",Node [Path [Leaf "W"],Path [Leaf "SSSEESEEEEESESWWSEESESWWWWWNE",Node [Path [Leaf "EE"],Path [Leaf "NWWWSWNWWSSWSESENESSSSENNESSSWWSESSEENWNEEEESESSSSEENESSWSEESWSESESENEESWSWWSWNNWWSSSWNWWWWSWNWNWSSESWWSESWSWSEEEN",Node [Path [Leaf "W"],Path [Leaf "EESWSSSWWNN",Node [Path [Leaf "ESNW"],Path []],Leaf "WSSWNNWNWNENNNE",Node [Path [Leaf "SS"],Path [Leaf "NNNE",Node [Path [Leaf "NNNNEESSES",Node [Path [Leaf "WWNNSSEE"],Path []],Leaf "SEENENEE",Node [Path [Leaf "E",Node [Path [Leaf "E"],Path [Leaf "NWN",Node [Path [Leaf "WNWSS",Node [Path [Leaf "WW",Node [Path [Leaf "S",Node [Path [Leaf "S"]

In [7]:
{-# LANGUAGE BangPatterns #-}

import qualified Data.Map.Strict as Map

move :: (Int, Int) -> Char -> (Int, Int)
move (x,y) c = case c of
    'N' -> (x, y+1)
    'W' -> (x-1, y)
    'E' -> (x+1, y)
    'S' -> (x, y-1)
    
visitString
    :: (Map.Map (Int, Int) Int)
    -> (Int, Int)
    -> Int
    -> String
    -> (Map.Map (Int, Int) Int, (Int, Int), Int)
visitString rooms curPos doorCount str = case str of
    [] -> (rooms, curPos, doorCount)
    _ -> update rooms curPos doorCount str
    where
        update rs pos count s = case s of
            [] -> (rs, pos, count)
            x:xs -> let
                pos' = move pos x
                count' = count + 1
                !rs' = Map.insertWith min pos' count' rs
                in update rs' pos' count' xs

In [8]:
import Debug.Trace

visitTerms :: Map.Map (Int, Int) Int -> (Int, Int) -> Int -> [Segment String] -> [(Map.Map (Int, Int) Int, (Int, Int), Int)]
visitTerms !rooms curPos doorCount [] = [(rooms, curPos, doorCount)]
visitTerms !rooms curPos doorCount (Leaf str:ts) = traceShow (Map.size rooms) $ let
    (rooms', curPos', doorCount') = visitString rooms curPos doorCount str
    in visitTerms rooms' curPos' doorCount' ts
visitTerms !rooms curPos doorCount (Node rs:ts) = traceShow (Map.size rooms) $ do
        Path r <- rs
        visitTerms rooms curPos doorCount (r++ts)

In [9]:
visitGraph (Path ts) =
    Map.unionsWith min $ map (\(rs,_,_) -> rs) $ visitTerms Map.empty (0,0) 0 ts

In [10]:
visitGraph' (Path ts) = visitTerms Map.empty (0,0) 0 ts

visitGraph' processed

In [33]:
Path ps = processed
length ps
head ps
Node bs = head $ tail ps
head bs

2

Leaf "WSSSEESESWSWSSWNNNENWWWWNNWWNWSSWNNNEEENEESE"

Path [Leaf "NENEENEESENENEEESWWSWSEENEESENNWNENENNEESSW",Node [Path [Leaf "SWSESSW",Node [Path [Leaf "N"],Path [Leaf "SESSWSSEEENNEESSENESEENNW",Node [Path [Leaf "NNWWS",Node [Path [Leaf "ESWENW"],Path []],Leaf "WWNNNENWWN",Node [Path [Leaf "EEENEEEENENWNENESEESSESWWSESENENENWNWNNESENNNNEEEENNNWNNESESSESSSW",Node [Path [Leaf "WWWWSEESWWSS",Node [Path [Leaf "W"],Path [Leaf "SSSEESEEEEESESWWSEESESWWWWWNE",Node [Path [Leaf "EE"],Path [Leaf "NWWWSWNWWSSWSESENESSSSENNESSSWWSESSEENWNEEEESESSSSEENESSWSEESWSESESENEESWSWWSWNNWWSSSWNWWWWSWNWNWSSESWWSESWSWSEEEN",Node [Path [Leaf "W"],Path [Leaf "EESWSSSWWNN",Node [Path [Leaf "ESNW"],Path []],Leaf "WSSWNNWNWNENNNE",Node [Path [Leaf "SS"],Path [Leaf "NNNE",Node [Path [Leaf "NNNNEESSES",Node [Path [Leaf "WWNNSSEE"],Path []],Leaf "SEENENEE",Node [Path [Leaf "E",Node [Path [Leaf "E"],Path [Leaf "NWN",Node [Path [Leaf "WNWSS",Node [Path [Leaf "WW",Node [Path [Leaf "S",Node [Path [Leaf "S"],Path [Leaf "E"]]],Path [Leaf "NW",Node [Path [Leaf "NENWNNN",No